In [1]:
!pip install evaluate datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.1 MB/s eta 0:00:00


In [2]:
import os
import json

import numpy as np

import warnings
import os

from transformers import (GPT2LMHeadModel,
                          GPT2Tokenizer,
                          TrainingArguments,
                          Trainer,
                          DataCollatorForLanguageModeling,
                          TextDataset,
                          pipeline)

from datasets import load_dataset
from evaluate import load

from tqdm import tqdm

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

warnings.filterwarnings("ignore")

In [3]:
model_name_or_path = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Попробуем сгенерировать стихотворение по первой строчке с претрейн моделью

In [4]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "В понедельник"
generator(prompt)

[{'generated_text': 'В понедельник, в среду, в 19.00 в зале заседаний администрации города состоится заседание комиссии по вопросам'}]

In [5]:
dataset = load_dataset("dkagramanyan/horoscopes_ru")
dataset

Generating train split:   0%|          | 0/66501 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6976 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 66501
    })
    test: Dataset({
        features: ['date', 'sign', 'text'],
        num_rows: 6976
    })
})

In [6]:
dataset['train'][0]

{'date': '2004-01-01',
 'sign': 'овен',
 'text': 'Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.'}

Запустим инференс на претрейн модельке

In [7]:
def preprocess_function(sample):
    return tokenizer(sample["text"])

tokenized_ds = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/66501 [00:00<?, ? examples/s]

Map:   0%|          | 0/6976 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    output_dir = "./gptru_small_10ep",
    overwrite_output_dir = True,
    num_train_epochs = 1,
    fp16 = True,
    per_device_train_batch_size = 8,
    learning_rate = 2e-5,
    lr_scheduler_type = 'cosine',
    report_to='tensorboard'
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

In [13]:
perplexity = load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return perplexity.compute(predictions=predictions, model_id='gpt2')

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [15]:
trainer.train()

Step,Training Loss
500,3.232400
1000,3.066900
1500,2.969300
2000,2.919700
2500,2.852000
3000,2.816000
3500,2.781600
4000,2.738400
4500,2.720000
5000,2.709700


KeyboardInterrupt: 

In [ ]:
val_results = trainer.evaluate()
val_results

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100)

prompt = "Я помню чудное мгновенье "
print(generator(prompt)[0]['generated_text'])
